# Tests of the various re-alignment techniques

In [2]:
import importlib
import numpy as np
import sys
import torch

sys.path.insert(0, '../../')
import src.align
importlib.reload(src.align)
from src.align import (BottomUpAlignmentMethod,
    TopDownAlignmentMethod,
    TopDownWeightMatchingBasedAlignment,
    WeightSortingBasedAlignment,
    BottomUpWeightMatchingBasedAlignment, 
    BottomUpActivationMatchingBasedAlignment,
    TopDownActivationMatchingBasedAlignment,
    BottomUpCorrelationMatchingBasedAlignment,
    GreedyMatching, 
    HungarianAlgorithmMatching)
from src.models import CNNLarge, GenericMLP

torch.set_printoptions(precision=10)

In [3]:
torch.manual_seed(0)
records = torch.normal(size=(5, 3, 32, 32), mean=0, std=1)
records_flattened = torch.flatten(records, 1)
print(records_flattened.size())

torch.Size([5, 3072])


We first test that re-alignment works for the identity permutation. We test that the outputs of the model remain the same.

In [4]:
torch.manual_seed(0)
model = CNNLarge(3)
model.eval()

print('Model layer sizes', model.conv1[0].weight.size(), model.conv2[0].weight.size(), model.fc1[0].weight.size())

print(model(records))

a1 = BottomUpAlignmentMethod()
aligned_model = a1.align_layers(model, sorted_idxs={'conv1': np.arange(20), 'conv2': np.arange(50), 'fc1': np.arange(500)})

print(aligned_model(records))

Model layer sizes torch.Size([20, 3, 5, 5]) torch.Size([50, 20, 5, 5]) torch.Size([500, 1250])
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)


Testing the Generic MLP. The output should stay the same whether we apply the identity permutation or a different permutation. The weights should stay the same when we apply the identity permutation but should be permuted accordingly when we apply a different permutation.

In [6]:
torch.manual_seed(0)
model = GenericMLP(layer_sizes=(3072, 7, 6, 5, 4, 3, 2))
model.eval()

#print(model)

print(model(records_flattened))
print(model.fc1.linear.bias[:3])
print(model.fc2.linear.weight[0, :3])

a1 = BottomUpAlignmentMethod()
aligned_model = a1.align_layers(model, sorted_idxs={'fc1': np.arange(7), 'fc2': np.arange(6), 'fc3': np.arange(5),\
    'fc4': np.arange(4), 'fc5': np.arange(3)})

print(aligned_model(records_flattened))
print(aligned_model.fc1.linear.bias[:3])
print(aligned_model.fc2.linear.weight[0, :3])

permutation = np.arange(7)
permutation[0] = 1
permutation[1] = 0
aligned_model = a1.align_layers(model, sorted_idxs={'fc1': permutation, 'fc2': np.arange(6), 'fc3': np.arange(5),\
    'fc4': np.arange(4), 'fc5': np.arange(3)})
print(aligned_model.fc1.linear.bias[:3])
print(aligned_model.fc2.linear.weight[0, :3])

print(aligned_model(records_flattened))

tensor([[-0.0292731617, -0.5151621699],
        [-0.0294966158, -0.5110382438],
        [-0.0295566171, -0.5099309087],
        [-0.0296777636, -0.5076951385],
        [-0.0292622689, -0.5153631568]], grad_fn=<AddmmBackward0>)
tensor([-0.0113725066,  0.0048179408, -0.0048890654], grad_fn=<SliceBackward0>)
tensor([0.1931403130, 0.1464328617, 0.2207494080], grad_fn=<SliceBackward0>)
tensor([[-0.0292731617, -0.5151621699],
        [-0.0294966158, -0.5110382438],
        [-0.0295566171, -0.5099309087],
        [-0.0296777636, -0.5076951385],
        [-0.0292622689, -0.5153631568]], grad_fn=<AddmmBackward0>)
tensor([-0.0113725066,  0.0048179408, -0.0048890654], grad_fn=<SliceBackward0>)
tensor([0.1931403130, 0.1464328617, 0.2207494080], grad_fn=<SliceBackward0>)
tensor([ 0.0048179408, -0.0113725066, -0.0048890654], grad_fn=<SliceBackward0>)
tensor([0.1464328617, 0.1931403130, 0.2207494080], grad_fn=<SliceBackward0>)
tensor([[-0.0292731617, -0.5151621699],
        [-0.0294966158, -0.51103824

We test that re-alignment works for custom, user-defined permutations.

In [7]:
torch.manual_seed(0)
np.random.seed(0)

model = CNNLarge(3)
model.eval()

print('Model layer sizes', model.conv1[0].weight.size(), model.conv2[0].weight.size(), model.fc1[0].weight.size())

print(model(records))

a2 = BottomUpAlignmentMethod()
print('Applying a permutation to the first layer only.')
permutation = np.random.permutation(20)
aligned_model = a2.align_layers(model, sorted_idxs={'conv1': permutation, 'conv2': np.arange(50), 'fc1': np.arange(500)})

print(aligned_model(records))

print(permutation, model.conv1[0].weight[:, 0, 3, 0], aligned_model.conv1[0].weight[:, 0, 3, 0])
print(permutation, model.conv1[0].bias, aligned_model.conv1[0].bias)

a3 = BottomUpAlignmentMethod()
print('Applying a permutation to the second layer only.')
aligned_model = a3.align_layers(model, sorted_idxs={'conv1': np.arange(20), 'conv2': np.random.permutation(50), 'fc1': np.arange(500)})

print(aligned_model(records))

a4 = BottomUpAlignmentMethod()
print('Applying a permutation to the third layer only.')
aligned_model = a4.align_layers(model, sorted_idxs={'conv1': np.arange(20), 'conv2': np.arange(50), 'fc1': np.random.permutation(500)})

print(aligned_model(records))

a5 = BottomUpAlignmentMethod()
print('Applying a permutation to all the layers.')
aligned_model = a5.align_layers(model, sorted_idxs={'conv1': np.random.permutation(20), 'conv2': np.random.permutation(50), 'fc1': np.random.permutation(500)})

print(aligned_model(records))

Model layer sizes torch.Size([20, 3, 5, 5]) torch.Size([50, 20, 5, 5]) torch.Size([500, 1250])
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
Applying a permutation to the first layer only.
tensor([[ 0.0742715523,  0.1782564074,  0.0600745901],
        [ 0.1083159596,  0.1627133787,  0.0658591017],
        [ 0.1604914814,  0.2382882684, -0.0252837837],
        [ 0.0827142075,  0.1198018044,  0.0181518812],
        [ 0.1977792531,  0.2047205418, -0.0100920498]],
       grad_fn=<AddmmBackward0>)
[18  1 19  8 10 17  6 13  4  2  5 14  9  7 16 11  3  0 15 12] tensor([ 0.0042774230,  0.0423303209,  0.0014932797,  0.0381272994,
         0.0768987313,  0.0132682649,  0.0619851127, -0.0603320338,
         0.1004339606,  0.0463842377, -0.

Check that the permutations work as expected.

In [8]:
torch.manual_seed(0)

model = CNNLarge(3)
model.eval()

print('Model layer sizes', model.conv1[0].weight.size(), model.conv2[0].weight.size(), model.fc1[0].weight.size())

print(model(records))
print(model.conv1[0].weight[:, 0, 0, 0])

a6 = BottomUpAlignmentMethod()
permutation = np.arange(20)
permutation[1] = 0
permutation[0] = 1
aligned_model = a6.align_layers(model, sorted_idxs={'conv1': permutation, 'conv2': np.arange(50), 'fc1': np.arange(500)})

print(aligned_model(records))
print(model.conv1[0].weight[:, 0, 0, 0], aligned_model.conv1[0].weight[:, 0, 0, 0])
print(model.conv2[0].weight[0, :, 0, 0], aligned_model.conv2[0].weight[0, :, 0, 0])

Model layer sizes torch.Size([20, 3, 5, 5]) torch.Size([50, 20, 5, 5]) torch.Size([500, 1250])
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
tensor([-0.0008645035,  0.1018857807, -0.1080628857,  0.0572013110,
        -0.0557106063, -0.0655576065,  0.0957626626, -0.0586115047,
        -0.0974088088,  0.0886598602, -0.0928521156, -0.0195688419,
         0.0966262743,  0.0099808285, -0.0217446089, -0.0162298270,
         0.0490856394, -0.0343011618, -0.0517730713,  0.0110124676],
       grad_fn=<SelectBackward0>)
tensor([[ 0.0742716193,  0.1782564223,  0.0600745529],
        [ 0.1083159745,  0.1627133787,  0.0658591986],
        [ 0.1604915261,  0.2382883430, -0.0252837166],
        [ 0.0827142149,  0.1198017746,  0.0181517489],
 

We test that the weight sorting-based alignment works as expected.

In [9]:
torch.manual_seed(0)

model = CNNLarge(3)
model.eval()

print('Model layer sizes', model.conv1[0].weight.size(), model.conv2[0].weight.size(), model.fc1[0].weight.size())

print(model(records))

a7 = WeightSortingBasedAlignment()
aligned_model = a7.align_layers(model)

print(aligned_model(records))
print('Order before', model.conv1[0].weight.view(model.conv1[0].weight.size(0), -1).sum(dim=1))
print('Order after', aligned_model.conv1[0].weight.view(aligned_model.conv1[0].weight.size(0), -1).sum(dim=1))

print('Order before', model.fc1.linear.weight.view(model.fc1.linear.weight.size(0), -1).sum(dim=1)[:10])
print('Order after', aligned_model.fc1.linear.weight.view(aligned_model.fc1.linear.weight.size(0), -1).sum(dim=1)[:10])

Model layer sizes torch.Size([20, 3, 5, 5]) torch.Size([50, 20, 5, 5]) torch.Size([500, 1250])
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0742715597,  0.1782563776,  0.0600745790],
        [ 0.1083160043,  0.1627133638,  0.0658590943],
        [ 0.1604915559,  0.2382882684, -0.0252837464],
        [ 0.0827142298,  0.1198018342,  0.0181518402],
        [ 0.1977792233,  0.2047205418, -0.0100920498]],
       grad_fn=<AddmmBackward0>)
Order before tensor([-0.0668168366, -0.5240811110, -0.7045913935, -0.3533548117,
         0.4629522562,  0.4145592451,  0.3347054124, -0.0400037766,
        -0.1369437724,  0.2329905033,  0.1089634970,  0.4734973907,
         0.0717745423, -0.3839237094, -0.7761198878, -0.9724586010,
  

Same test for the GenericMLP.

In [10]:
torch.manual_seed(0)
model = GenericMLP(layer_sizes=(3072, 7, 6, 5, 4, 3, 2))
model.eval()

#print(model)

print(model(records_flattened))

a7 = WeightSortingBasedAlignment()
aligned_model = a7.align_layers(model)

print(aligned_model(records_flattened))
print('Order before', model.fc1.linear.weight.view(7, -1).sum(dim=1))
print('Order after', aligned_model.fc1.linear.weight.view(7, -1).sum(dim=1))

print('Order before', model.fc2.linear.weight.view(6, -1).sum(dim=1))
print('Order after', aligned_model.fc2.linear.weight.view(6, -1).sum(dim=1))

tensor([[-0.0292731617, -0.5151621699],
        [-0.0294966158, -0.5110382438],
        [-0.0295566171, -0.5099309087],
        [-0.0296777636, -0.5076951385],
        [-0.0292622689, -0.5153631568]], grad_fn=<AddmmBackward0>)
tensor([[-0.0292731617, -0.5151621699],
        [-0.0294966158, -0.5110382438],
        [-0.0295566171, -0.5099309087],
        [-0.0296777636, -0.5076951385],
        [-0.0292622689, -0.5153631568]], grad_fn=<AddmmBackward0>)
Order before tensor([-0.2159837186, -0.3019032180,  0.7937227488,  0.4051253796,
         0.1338184774,  0.5888521671, -0.0695017725], grad_fn=<SumBackward1>)
Order after tensor([-0.3019032180, -0.2159837186, -0.0695017725,  0.1338184774,
         0.4051253796,  0.5888521671,  0.7937227488], grad_fn=<SumBackward1>)
Order before tensor([ 0.0541683435, -0.3083331585,  0.5434048176,  0.2548643947,
        -0.4114866853,  0.6938853860], grad_fn=<SumBackward1>)
Order after tensor([-0.4114866853, -0.3083331585,  0.0541683435,  0.2548644543,
     

We test that the matching techniques work as expected.

In [11]:
greedy_matching = GreedyMatching()

hungarian_matching = HungarianAlgorithmMatching()

print(greedy_matching.match(torch.FloatTensor([
    [2, 1, 0.01],
    [3, 4, 5],
    [0.05, 0.1, 0.15]])))
    
print(hungarian_matching.match(torch.FloatTensor([
    [2, 1, 0.01],
    [3, 4, 5],
    [0.05, 0.1, 0.15]])))

print(greedy_matching.match(torch.FloatTensor([
    [0.1, 3, 1],
    [5, 4, 3.5],
    [4, 4, 3]])))

print(hungarian_matching.match(torch.FloatTensor([
    [0.1, 3, 1],
    [5, 4, 3.5],
    [4, 4, 3]])))

[2. 1. 0.]
[2 0 1]
[0. 1. 2.]
[0 1 2]


We test that weight matching-based re-alignment works.

In [12]:
torch.manual_seed(0)

model1, model2 = CNNLarge(3), CNNLarge(3)
model1.eval(), model2.eval()

print('Model layer sizes', model1.conv1[0].weight.size(), model1.conv2[0].weight.size(), model1.fc1[0].weight.size())

print(model1(records))

a8 = BottomUpWeightMatchingBasedAlignment(GreedyMatching())
aligned_model1 = a8.align_layers(model1, model2)

print(aligned_model1(records))
print(a8.elapsed_times)

a8bis = BottomUpWeightMatchingBasedAlignment(HungarianAlgorithmMatching())
a8bis.align_layers(model1, model2)

print(model1(records))
print(a8bis.elapsed_times)

Model layer sizes torch.Size([20, 3, 5, 5]) torch.Size([50, 20, 5, 5]) torch.Size([500, 1250])
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
I am here
tensor([[ 0.0742714852,  0.1782564223,  0.0600746609],
        [ 0.1083159596,  0.1627133489,  0.0658590943],
        [ 0.1604915857,  0.2382882833, -0.0252837464],
        [ 0.0827142000,  0.1198017746,  0.0181518532],
        [ 0.1977791190,  0.2047205269, -0.0100920610]],
       grad_fn=<AddmmBackward0>)
[0.00026345252990722656, 0.0006096363067626953, 0.026691436767578125]
I am here
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.11980

In [13]:
torch.manual_seed(0)
model1 = GenericMLP(layer_sizes=(3072, 70, 60, 50, 40, 30, 3))
model2 = GenericMLP(layer_sizes=(3072, 70, 60, 50, 40, 30, 3))
model1.eval(), model2.eval()

#print(model)

#print(model2(records_flattened))
print(model1(records_flattened))

a9 = BottomUpWeightMatchingBasedAlignment(HungarianAlgorithmMatching())
aligned_model1 = a9.align_layers(model1, model2)

print(aligned_model1(records_flattened))

a10 = BottomUpCorrelationMatchingBasedAlignment(HungarianAlgorithmMatching())
aligned_model2 = a10.align_layers(model1, model2, records=records_flattened)

print(aligned_model2(records_flattened))

tensor([[-0.0988133550, -0.0700098723, -0.1872031987],
        [-0.0984527767, -0.0642039329, -0.1895425171],
        [-0.0974958241, -0.0685146526, -0.1857692599],
        [-0.0992155895, -0.0657143146, -0.1859866083],
        [-0.0967054814, -0.0699071437, -0.1942999810]],
       grad_fn=<AddmmBackward0>)
I am here
tensor([[-0.0988133550, -0.0700098723, -0.1872031987],
        [-0.0984527767, -0.0642039478, -0.1895425171],
        [-0.0974958241, -0.0685146600, -0.1857692599],
        [-0.0992155895, -0.0657143220, -0.1859866083],
        [-0.0967054963, -0.0699071512, -0.1942999810]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0988133550, -0.0700098723, -0.1872031987],
        [-0.0984527767, -0.0642039478, -0.1895425171],
        [-0.0974958241, -0.0685146451, -0.1857692748],
        [-0.0992155895, -0.0657143220, -0.1859866083],
        [-0.0967054889, -0.0699071437, -0.1942999810]],
       grad_fn=<AddmmBackward0>)


We test that activation-based matching works.

In [14]:
torch.manual_seed(0)

model1, model2 = CNNLarge(3), CNNLarge(3)
model1.eval(), model2.eval()

print('Model layer sizes', model1.conv1[0].weight.size(), model1.conv2[0].weight.size(), model1.fc1[0].weight.size())

print(model1(records))
biases_before = set(model1.fc1.linear.bias.tolist())
print(model1.fc1.linear.bias[:10])

a10 = BottomUpActivationMatchingBasedAlignment(HungarianAlgorithmMatching())
#print(records)
aligned_model1 = a10.align_layers(model1, model2, records=records)

print(aligned_model1(records))
print(a10.elapsed_times)

biases_after = set(aligned_model1.fc1.linear.bias.tolist())

assert len(biases_before.intersection(biases_after)) == len(biases_after)
print(aligned_model1.fc1.linear.bias[:10])

print(model1.conv1[0].bias)
print(aligned_model1.conv1[0].bias)

Model layer sizes torch.Size([20, 3, 5, 5]) torch.Size([50, 20, 5, 5]) torch.Size([500, 1250])
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
tensor([-0.0063933060,  0.0175652150,  0.0109577179,  0.0020585060,
         0.0131788002, -0.0199355874,  0.0182594843,  0.0087319063,
         0.0174824577,  0.0227316059], grad_fn=<SliceBackward0>)
tensor([[ 0.0742715448,  0.1782563180,  0.0600745678],
        [ 0.1083159745,  0.1627133042,  0.0658590719],
        [ 0.1604914963,  0.2382882386, -0.0252837762],
        [ 0.0827142373,  0.1198018119,  0.0181519128],
        [ 0.1977791488,  0.2047205418, -0.0100920498]],
       grad_fn=<AddmmBackward0>)
[9.775161743164062e-05, 0.0001761913299560547, 0.02301645278930664]
tensor([ 0.0182594

# Testing the top-down alignment

Testing that the identity permutation works as expected.

In [15]:
torch.manual_seed(0)
model = CNNLarge(3)
model.eval()

print('Model layer sizes', model.fc1[0].weight.size(0), model.conv2[0].weight.size(0), model.conv1[0].weight.size(0))

print(model(records))

t1 = TopDownAlignmentMethod()
aligned_model = t1.align_layers(model, sorted_idxs={'fc2': np.arange(500), 'fc1': np.arange(50), 'conv2': np.arange(20)})

print(aligned_model(records))

Model layer sizes 500 50 20
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)


Testing that a custom permutation permutation works as expected.

In [16]:
torch.manual_seed(0)

model = CNNLarge(3)
model.eval()

print('Model layer sizes', model.conv1[0].weight.size(), model.conv2[0].weight.size(), model.fc1[0].weight.size())

print(model(records))

t2 = TopDownAlignmentMethod()
permutation = np.arange(500)
permutation[1] = 0
permutation[0] = 1
aligned_model = t2.align_layers(model, sorted_idxs={'fc2': permutation, 'fc1': np.arange(50), 'conv2': np.arange(20)})

print(aligned_model(records))
print(model.fc2.linear.weight[0, :4])
print(aligned_model.fc2.linear.weight[0, :4])

print(model.fc1.linear.weight[:4, 0])
print(aligned_model.fc1.linear.weight[:4, 0])

Model layer sizes torch.Size([20, 3, 5, 5]) torch.Size([50, 20, 5, 5]) torch.Size([500, 1250])
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0742715597,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591315],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920573]],
       grad_fn=<AddmmBackward0>)
tensor([-0.0216782764, -0.0295759179, -0.0382286869,  0.0249514319],
       grad_fn=<SliceBackward0>)
tensor([-0.0295759179, -0.0216782764, -0.0382286869,  0.0249514319],
       grad_fn=<SliceBackward0>)
tensor([ 0.0266703460,  0.0243966989, -0.0170943607, -0.0203894544],
       grad_fn

In [17]:
torch.manual_seed(0)
np.random.seed(0)

model = CNNLarge(3)
model.eval()

print('Model layer sizes', model.conv1[0].weight.size(), model.conv2[0].weight.size(), model.fc1[0].weight.size())

print(model(records))

t3 = TopDownAlignmentMethod()
aligned_model = t3.align_layers(model, sorted_idxs={'fc2': np.random.permutation(500), \
    'fc1': np.random.permutation(50), 'conv2': np.random.permutation(20)})

print(aligned_model(records))

Model layer sizes torch.Size([20, 3, 5, 5]) torch.Size([50, 20, 5, 5]) torch.Size([500, 1250])
tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0742715523,  0.1782563627,  0.0600746237],
        [ 0.1083159745,  0.1627133042,  0.0658590123],
        [ 0.1604914963,  0.2382882982, -0.0252837986],
        [ 0.0827142969,  0.1198018044,  0.0181518570],
        [ 0.1977792233,  0.2047205865, -0.0100920610]],
       grad_fn=<AddmmBackward0>)


In [18]:
torch.manual_seed(0)
model1 = GenericMLP(layer_sizes=(3072, 70, 60, 50, 40, 30, 3))
model2 = GenericMLP(layer_sizes=(3072, 70, 60, 50, 40, 30, 3))
model1.eval(), model2.eval()

#print(model)

#print(model2(records_flattened))
print(model1(records_flattened))

t2 = TopDownAlignmentMethod()
permutation = np.arange(30)
permutation[0] = 1
permutation[1] = 0
aligned_model1 = t2.align_layers(model1, sorted_idxs={'fc2': np.arange(70), 'fc3': np.arange(60), 'fc4': np.arange(50), \
    'fc5': np.arange(40), 'fc6': permutation})

print(aligned_model1(records_flattened))

aligned_model2 = t2.align_layers(model1, sorted_idxs={'fc2': np.random.permutation(70), 'fc3': np.random.permutation(60), \
    'fc4': np.random.permutation(50), 'fc5': np.random.permutation(40), 'fc6': np.random.permutation(30)})

print(aligned_model2(records_flattened))

aligned_model3 = TopDownWeightMatchingBasedAlignment(HungarianAlgorithmMatching()).align_layers(model1, model2)

print(aligned_model3(records_flattened))

aligned_model4 = TopDownActivationMatchingBasedAlignment(HungarianAlgorithmMatching()).align_layers(model1, model2, records=records_flattened)

print(aligned_model4(records_flattened))

tensor([[-0.0988133550, -0.0700098723, -0.1872031987],
        [-0.0984527767, -0.0642039329, -0.1895425171],
        [-0.0974958241, -0.0685146526, -0.1857692599],
        [-0.0992155895, -0.0657143146, -0.1859866083],
        [-0.0967054814, -0.0699071437, -0.1942999810]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0988133550, -0.0700098723, -0.1872031987],
        [-0.0984527767, -0.0642039329, -0.1895425171],
        [-0.0974958241, -0.0685146526, -0.1857692599],
        [-0.0992155895, -0.0657143146, -0.1859866083],
        [-0.0967054814, -0.0699071437, -0.1942999810]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0988133550, -0.0700098649, -0.1872032136],
        [-0.0984527767, -0.0642039329, -0.1895425171],
        [-0.0974958241, -0.0685146451, -0.1857692599],
        [-0.0992155895, -0.0657143220, -0.1859866083],
        [-0.0967054814, -0.0699071437, -0.1942999810]],
       grad_fn=<AddmmBackward0>)
I am here
tensor([[-0.0988133550, -0.0700098649, -0.1872031987],
        

In [19]:
torch.manual_seed(0)
np.random.seed(0)

model1, model2 = CNNLarge(3), CNNLarge(3)
model1.eval(), model2.eval()

print(model1(records))

t4 = TopDownWeightMatchingBasedAlignment(HungarianAlgorithmMatching())
aligned_model1 = t4.align_layers(model1, model2)

print(aligned_model1(records))
print(model1.fc2.linear.weight[0, :4])
print(aligned_model1.fc2.linear.weight[0, :4])

tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
I am here
tensor([[ 0.0742715746,  0.1782563925,  0.0600745454],
        [ 0.1083160043,  0.1627133936,  0.0658590943],
        [ 0.1604916006,  0.2382882386, -0.0252837539],
        [ 0.0827141851,  0.1198018417,  0.0181518570],
        [ 0.1977791339,  0.2047205120, -0.0100919977]],
       grad_fn=<AddmmBackward0>)
tensor([-0.0216782764, -0.0295759179, -0.0382286869,  0.0249514319],
       grad_fn=<SliceBackward0>)
tensor([-0.0282329880, -0.0384886377,  0.0438725203,  0.0271678474],
       grad_fn=<SliceBackward0>)


In [20]:
torch.manual_seed(0)
model1 = model = CNNLarge(3)
model2 = model = CNNLarge(3)
model1.eval(), model2.eval()

#print(model)

print(model1(records))

aligned_model1 = TopDownActivationMatchingBasedAlignment(HungarianAlgorithmMatching()).align_layers(model1, model2, records=records)

print(aligned_model1(records))
print(model1.conv1[0].bias)
print(aligned_model1.conv1[0].bias)

tensor([[ 0.0742715672,  0.1782564372,  0.0600745715],
        [ 0.1083159596,  0.1627133638,  0.0658591390],
        [ 0.1604915261,  0.2382883430, -0.0252837017],
        [ 0.0827141851,  0.1198017597,  0.0181517880],
        [ 0.1977791786,  0.2047205120, -0.0100920647]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0742715448,  0.1782563180,  0.0600745678],
        [ 0.1083159894,  0.1627133042,  0.0658590719],
        [ 0.1604914963,  0.2382882386, -0.0252837762],
        [ 0.0827142373,  0.1198018119,  0.0181519128],
        [ 0.1977791637,  0.2047205418, -0.0100920461]],
       grad_fn=<AddmmBackward0>)
Parameter containing:
tensor([ 0.0931478292, -0.0902533308,  0.0397556275, -0.0434939526,
        -0.0485915691, -0.0240835622, -0.0427407697,  0.0632729903,
        -0.0219392069,  0.0875514373,  0.0263483208, -0.0178418867,
        -0.0666954368, -0.1057328135, -0.0336851887, -0.1069289148,
        -0.0213932134,  0.0018872782, -0.0074413321, -0.0567208417],
       requires_gra